In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from global_model import GlobalModel
import pandas as pd
from supported_modles import Supported_modles
import utils
import numpy as np
from sklearn.metrics import f1_score
import random

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.MLP_classifier

In [4]:
clients, test_x, test_y = utils.set_data(True, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [5]:
f1_before = []
for client in clients:
    client.init_empty_model(selected_model,0.01)
    client.train_model()
    y_hat = client.model.predict(test_x)
    f1_before.append(f1_score(test_y,y_hat, average="binary"))
    client.split_data()
    print('.')

AttributeError: 'MLPClassifier' object has no attribute '_label_binarizer'

In [7]:
f1_before

[0.9947910429888637,
 0.9945059057770326,
 0.9952062797734966,
 0.9950877613370873,
 0.9956259923904252]

In [8]:
global_model = GlobalModel(selected_model)

In [9]:
number_of_rounds = 3
start = True
for _ in range(number_of_rounds):
    RANDOM_NUMBER_OF_CLIENTS = random.randint(1,4)

    print(f'Starting new round!')
    f1s = np.empty((0,RANDOM_NUMBER_OF_CLIENTS), float)
    data_weights = arr = np.empty((0,RANDOM_NUMBER_OF_CLIENTS), float)

    applicable_clients = random.sample((clients), RANDOM_NUMBER_OF_CLIENTS)

    if start:
        print('Initializing Global Model')
        global_model.model = applicable_clients[0].model
        start = False

    for client in applicable_clients:
        print(f'Client name: {client.name}')
        f1s = np.append(f1s, client.test_model_f1())
        data_weights = np.append(data_weights, client.x.shape[0])
        
    round_weights = (f1s/sum(f1s) + data_weights/sum(data_weights))/2
    print(round_weights)

    global_model.update_global_model(applicable_clients,round_weights,selected_model)

    for client in clients:
        f1_fede = global_model.f1_score(client.x_test,client.y_test)
        f1_local = client.test_model_f1()

        print(f1_local)
        print(f1_fede)

        if f1_fede > f1_local:
            print("f1 fede was better")
            client.F1 = f1_fede
            # local was better set to local values
            if selected_model == Supported_modles.MLP_classifier:
                client.model.intercepts_ = global_model.model.intercepts_
                client.model.coefs_ = global_model.model.coefs_
            else:
                client.model.intercept_ = global_model.model.intercept_
                client.model.coef_ = global_model.model.coef_
                
    print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Starting new round!
Initializing Global Model
Client name: node3
[1.]


/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


0.9956659924877203
0.9956627700441985
0.9950426994376171
0.995704574836315
f1 fede was better
0.9960553087239963
0.9960553087239963
0.9961816219805761
0.9958492445625104
0.996120992035654
0.995316787357389
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node1
Client name: node4
Client name: node5
Client name: node3
[0.22495733 0.22502204 0.32501443 0.22500619]


/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


0.9956659924877203
0.7851464224615076
0.995704574836315
0.7834495394167642
0.7766792375904596
0.7766792375904596
0.9961816219805761
0.7834143377885783
0.996120992035654
0.7837334541245157
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Starting new round!
Client name: node5
[1.]


/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


0.9956659924877203
0.995910949568378
f1 fede was better
0.995704574836315
0.9961628294961629
f1 fede was better
0.9960546534324516
0.9960546534324516
0.9961816219805761
0.9961399576640518
0.996120992035654
0.996120992035654
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [10]:
i = 0
for client in clients:
    y_hat = client.model.predict(test_x)
    print(f1_score(test_y,y_hat) - f1_before[i])
    
    i += 1

0.00083494940156148
0.001120086613392579
0.0004197126169285248
0.0
0.0
